In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [42]:
## Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

RowNumber  CustomerId   Surname  ...  IsActiveMember EstimatedSalary Exited
0          1    15634602  Hargrave  ...               1       101348.88      1
1          2    15647311      Hill  ...               1       112542.58      0
2          3    15619304      Onio  ...               0       113931.57      1
3          4    15701354      Boni  ...               0        93826.63      0
4          5    15737888  Mitchell  ...               1        79084.10      0

[5 rows x 14 columns]

In [43]:
# Preprocessing the data
## Drop the unnecessary columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1) #axis=1 means we are dropping columns

In [44]:
data

CreditScore Geography  Gender  ...  IsActiveMember  EstimatedSalary  Exited
0             619    France  Female  ...               1        101348.88       1
1             608     Spain  Female  ...               1        112542.58       0
2             502    France  Female  ...               0        113931.57       1
3             699    France  Female  ...               0         93826.63       0
4             850     Spain  Female  ...               1         79084.10       0
...           ...       ...     ...  ...             ...              ...     ...
9995          771    France    Male  ...               0         96270.64       0
9996          516    France    Male  ...               1        101699.77       0
9997          709    France  Female  ...               1         42085.58       1
9998          772   Germany    Male  ...               0         92888.52       1
9999          792    France  Female  ...               0         38190.78       0

[10000 rows x 11 columns]

In [45]:
## encode the categorical variables
lable_encoder_gender = LabelEncoder()
data['Gender'] = lable_encoder_gender.fit_transform(data['Gender']) #fit_transform is used to fit and transform the data at the same time
data

CreditScore Geography  Gender  ...  IsActiveMember  EstimatedSalary  Exited
0             619    France       0  ...               1        101348.88       1
1             608     Spain       0  ...               1        112542.58       0
2             502    France       0  ...               0        113931.57       1
3             699    France       0  ...               0         93826.63       0
4             850     Spain       0  ...               1         79084.10       0
...           ...       ...     ...  ...             ...              ...     ...
9995          771    France       1  ...               0         96270.64       0
9996          516    France       1  ...               1        101699.77       0
9997          709    France       0  ...               1         42085.58       1
9998          772   Germany       1  ...               0         92888.52       1
9999          792    France       0  ...               0         38190.78       0

[10000 rows x 11 columns]

In [46]:
## Onehot encode 'Geography' column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [47]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [48]:
geo_enoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography'])) # toarray() is used to convert the sparse matrix to a dense matrix
geo_enoded_df

Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]

In [49]:
## Combine onehot encoded columns with the original dataframe
data=pd.concat([data.drop('Geography', axis=1), geo_enoded_df], axis=1)
data.head()

CreditScore  Gender  ...  Geography_Germany  Geography_Spain
0          619       0  ...                0.0              0.0
1          608       0  ...                0.0              1.0
2          502       0  ...                0.0              0.0
3          699       0  ...                0.0              0.0
4          850       0  ...                0.0              1.0

[5 rows x 13 columns]

In [33]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(lable_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)


In [34]:
## Divide the dataset into independent and dependent features
X = data.drop('Exited', axis=1) # X is the independent feature
y = data['Exited'] # y is the dependent feature

In [35]:
## Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [36]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [37]:
with open ('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Implmentaion

In [38]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [39]:
## import ANN Model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 Connected with input layer
    Dense(32, activation='relu'), ## HL2
    Dense(1, activation='relu'), ## Output layer
])

f:\NewPersonalProjects2025\Machine_Learnings\Python\ANNClassification\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [40]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x000001DEB2480EA0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [53]:
## Compile the model
model.compile(optimizer=opt, loss=loss, metrics=['accuracy']) ## adam is the optimizer have fixed learning rate

In [75]:
## Setup the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [76]:
## Set up Early stopping bcs of overfitting
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [77]:
### Train the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7948 - loss: 3.3078 - val_accuracy: 0.8035 - val_loss: 3.1672
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7887 - loss: 3.4058 - val_accuracy: 0.8035 - val_loss: 3.1672
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7945 - loss: 3.3115 - val_accuracy: 0.8035 - val_loss: 3.1672
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7919 - loss: 3.3543 - val_accuracy: 0.8035 - val_loss: 3.1672
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7888 - loss: 3.4037 - val_accuracy: 0.8035 - val_loss: 3.1672
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7972 - loss: 3.2685 - val_accuracy: 0.8035 - val_loss: 3.1672
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7921 - loss: 3.3514 - val_accuracy: 0.8035 - val_loss: 3.1672
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7922 - loss: 3.3494 - val_accu

In [78]:
model.save('model.h5')

In [79]:
## Load Tensorboard Extension
%load_ext tensorboard 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [80]:
## %tensorboard --logdir logs/fit20250718-133920
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 17464), started 0:09:43 ago. (Use '!kill 17464' to kill it.)

In [ ]:
## Load the Pickel File